**Work in progress !!**

***Task Completed :***

*   Architecture
*   Train VAE
*   Implement ELBO
*   Achieve an ELBO of ≥−96
*   Returns:–(logp(x1),...,logp(xM)) estimates of size (M,)

***To Do:***
* Nothing





In [ ]:
from torchvision.datasets import utils
import torch.utils.data as data_utils
import torch
import os
import numpy as np
from torch import nn
from torch.nn.modules import upsampling
from torch.functional import F
from torch.optim import Adam
from torch.autograd import Variable
import tensorflow as tf


In [ ]:
def get_data_loader(dataset_location, batch_size):
    URL = "http://www.cs.toronto.edu/~larocheh/public/datasets/binarized_mnist/"
    # start processing
    def lines_to_np_array(lines):
        return np.array([[int(i) for i in line.split()] for line in lines])
    splitdata = []
    for splitname in ["train", "valid", "test"]:
        filename = "binarized_mnist_%s.amat" % splitname
        filepath = os.path.join(dataset_location, filename)
        utils.download_url(URL + filename, dataset_location, filename = filename, md5=None)
        with open(filepath) as f:
            lines = f.readlines()
        x = lines_to_np_array(lines).astype('float32')
        x = x.reshape(x.shape[0], 1, 28, 28)
        # pytorch data loader
        dataset = data_utils.TensorDataset(torch.from_numpy(x))
        dataset_loader = data_utils.DataLoader(x, batch_size=batch_size, shuffle=splitname == "train")
        splitdata.append(dataset_loader)
    return splitdata

In [ ]:
epochs = 20
bs = 128

train, valid, test = get_data_loader("binarized_mnist", bs)

  0%|          | 16384/78400000 [00:00<09:18, 140325.16it/s]

78405632it [00:30, 5701191.78it/s]                              
0it [00:00, ?it/s]
  0%|          | 49152/15680000 [00:00<00:33, 461126.14it/s]


 98%|█████████▊| 15302656/15680000 [00:02<00:00, 5841153.05it/s]
15687680it [00:02, 5783540.86it/s]                              
0it [00:00, ?it/s]
  0%|          | 49152/15680000 [00:00<00:34, 453997.46it/s]


 98%|█████████▊| 15441920/15680000 [00:02<00:00, 6229514.18it/s]
15687680it [00:02, 6641940.90it/s]                              

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
for x in train:
    plt.imshow(x[0, 0])
    break

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


class UnFlatten(nn.Module):
    def forward(self, input, size=256):
        return input.view(input.size(0), size, 1, 1)

class VAE(nn.Module):
  
    def __init__(self, image_channels=1, h_dim=256, z_dim=100):
        super(VAE, self).__init__()
        
        # Q(z|X) -- encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=3),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 256, kernel_size=5),
            nn.ELU(),
            Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        # P(X|z) -- decoder
        self.decoder = nn.Sequential(
            
            UnFlatten(),
            nn.ELU(),
            nn.Conv2d(256, 64, kernel_size= 5, padding= 4),
            nn.ELU(),
            nn.Upsample(scale_factor=2, mode = 'bilinear', align_corners=True), #, align_corners=True
            nn.Conv2d(64, 32, kernel_size=3, padding=2),
            nn.ELU(),
            nn.Upsample(scale_factor=2, mode = 'bilinear', align_corners=True), #, align_corners=True
            nn.Conv2d(32, 16, kernel_size=3, padding = 2),
            nn.ELU(),
            nn.Conv2d(16, image_channels, kernel_size = 3, padding = 2),
            nn.Sigmoid(),
        )
        
        
    def reparameterize(self, mu, logvar):
      
        """std = logvar.mul(0.5).exp_()
        esp = torch.randn(*mu.size())
        z = mu + std * esp"""
        eps = torch.randn(mu.size()).cuda()
        z = eps.mul(logvar.mul(0.5).exp_()).add_(mu)
          
        logq_xz = torch.distributions.MultivariateNormal(mu,  torch.eye(100).cuda())
        q_xz = logq_xz.log_prob(z)
        
        log_p_z = torch.distributions.MultivariateNormal(torch.zeros(100).cuda(), torch.eye(100).cuda())
        p_z = log_p_z.log_prob(z)
                
        return z, q_xz, p_z

    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z, q_xz, p_z = self.reparameterize(mu, logvar)
        return z, q_xz, p_z, mu, logvar

    def forward(self, x):
      
        h = self.encoder(x)
        z, q_xz, p_z, mu, logvar = self.bottleneck(h)
        z_decoder = self.fc3(z)
        decoder = self.decoder(z_decoder)
        
        return decoder, mu, logvar
      
    def imp_sample(self, x, h):
        
        z, q_xz, p_z, mu, logvar = self.bottleneck(h)
        
        z_decoder = self.fc3(z)
        decoder = self.decoder(z_decoder)
        
        log_p_xz = torch.distributions.Bernoulli(decoder.view(decoder.size(0),784))
        p_xz = log_p_xz.log_prob(x.view(x.size(0),784))
        
        return p_xz, q_xz, p_z

        


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

model = VAE().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)


criterion = nn.BCELoss(reduction = "sum").cuda()

def loss_fn(recon_x, x, mu, logvar):
    ## E[log P(X|z)]
    
    BCE = criterion(recon_x, x)
    KLD = - 0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp() )
    return BCE + KLD, BCE, KLD


In [ ]:
epochs = 20

for epoch in range(epochs):
    total_loss = 0
    total_images = 0
    total_bce = 0
    total_kld = 0 
    for idx, (images) in enumerate(train):
        images = images.to(device)
        recon_images, mu, logvar = model(images)
        loss, bce, kld = loss_fn(recon_images, images, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
        total_bce += bce.data
        total_kld += kld.data
        total_images += images.size(0)
        
    to_print = "Epoch[{}/{}] ELBO: {:.3f} ,BCE: {:.3f} ,KLD: {:.3f}".format(epoch+1, 
                            epochs, -total_loss/total_images, total_bce/total_images, total_kld/total_images)
    print("Training Data : ", to_print)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Training Data :  Epoch[1/20] ELBO: -210.933 ,BCE: 196.092 ,KLD: 14.841
Training Data :  Epoch[2/20] ELBO: -140.425 ,BCE: 118.298 ,KLD: 22.127
Training Data :  Epoch[3/20] ELBO: -121.912 ,BCE: 98.859 ,KLD: 23.053
Training Data :  Epoch[4/20] ELBO: -113.155 ,BCE: 89.314 ,KLD: 23.841
Training Data :  Epoch[5/20] ELBO: -108.607 ,BCE: 84.276 ,KLD: 24.332
Training Data :  Epoch[6/20] ELBO: -105.808 ,BCE: 81.057 ,KLD: 24.751
Training Data :  Epoch[7/20] ELBO: -104.029 ,BCE: 78.970 ,KLD: 25.059
Training Data :  Epoch[8/20] ELBO: -102.484 ,BCE: 77.206 ,KLD: 25.278
Training Data :  Epoch[9/20] ELBO: -101.230 ,BCE: 75.737 ,KLD: 25.493
Training Data :  Epoch[10/20] ELBO: -100.229 ,BCE: 74.586 ,KLD: 25.642
Training Data :  Epoch[11/20] ELBO: -99.372 ,BCE: 73.607 ,KLD: 25.764
Training Data :  Epoch[12/20] ELBO: -98.727 ,BCE: 72.862 ,KLD: 25.865
Training Data :  Epoch[13/20] ELBO: -98.081 ,BCE: 72.154 ,KLD: 25.926
Training Data :  Epoch[14/20] ELBO: -97.564 ,BCE: 71.536 ,KLD: 26.027
Training Data :  

In [ ]:
# Q(z|X) -- encoder
# P(X|z) -- decoder
#p(z) -- latent


In [ ]:
#ELBO Validation Data:


epochs = 1


for epoch in range(epochs):
  
    total_loss = 0
    total_images = 0
    total_bce = 0
    total_kld = 0 
    for idx, (images) in enumerate(valid):
        images = images.to(device)
        recon_images, mu, logvar = model(images)
        loss, bce, kld = loss_fn(recon_images, images, mu, logvar)

        total_loss += loss.data
        total_bce += bce.data
        total_kld += kld.data
        total_images += images.size(0)
        
    to_print = "Epoch[{}/{}] ELBO: {:.3f} ,BCE: {:.3f} ,KLD: {:.3f}".format(epoch+1, 
                            epochs, -total_loss/total_images, total_bce/total_images, total_kld/total_images)
    print("Validation Data : ", to_print)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Validation Data :  Epoch[1/1] ELBO: -95.262 ,BCE: 69.002 ,KLD: 26.260


In [ ]:
#ELBO Test Data:

for epoch in range(epochs):
  
    total_loss = 0
    total_images = 0
    total_bce = 0
    total_kld = 0 
    for idx, (images) in enumerate(test):
        images = images.to(device)
        recon_images, mu, logvar = model(images)
        loss, bce, kld = loss_fn(recon_images, images, mu, logvar)

        total_loss += loss.data
        total_bce += bce.data
        total_kld += kld.data
        total_images += images.size(0)
        
    to_print = "Epoch[{}/{}] ELBO: {:.3f} ,BCE: {:.3f} ,KLD: {:.3f}".format(epoch+1, 
                            epochs, -total_loss/total_images, total_bce/total_images, total_kld/total_images)
    print("Test Data : ", to_print)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Test Data :  Epoch[1/1] ELBO: -94.578 ,BCE: 68.437 ,KLD: 26.141


In [ ]:
#Importance Sampling Function:

def importance_sampling(model, data):
    #Input as our trained model and data
  
    ip_loss = []
    count = 0
    #Setting the model parameters to no_grad
    for param in model.parameters():
        param.requires_grad = False

    for idx, (images) in enumerate(valid):
        images = images.to(device)
        count += 1
        #Calling model to get the output from encoder
        h = model.encoder(images.view(images.size(0),1,28,28))
        h = h.view(images.size(0),256)

        ip_sample = []

        for k in range(200):
            #Calling model to get output of our densities :
            p_xz, q_xz, p_z = model.imp_sample(images, h)
            #Summing over 784 dimention and the dimention we will have (1XBatch_Size)
            p_xz = torch.sum(p_xz,dim=1)
            #Importance Sampling calculation with LogSumExp trick:
            out = p_xz - q_xz + p_z
            log_weight = out - torch.max(out, 0)[0]
            weight = torch.exp(log_weight)
            weight = weight / torch.sum(weight, 0)
            loss = torch.mean(torch.sum(weight * (p_z + p_xz - q_xz), 0))
            ip_sample.append(loss)

        ip_loss.append(torch.sum(torch.stack(ip_sample))/bs)

        print("Importance Sampling over mini batch -", count,' : ' , torch.stack(ip_loss))

    log_likelihood_estimate = torch.mean(torch.stack(ip_loss))
    print('log-likelihood  estimate :', log_likelihood_estimate)
    
        

In [ ]:
#Importance Sampling on Validation Data:
importance_sampling(model, valid)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


In [ ]:
#Importance Sampling on Test Data:
importance_sampling(model, test)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Importance Sampling over mini batch - 79  :  tensor([-52.7593, -40.0179, -41.9021, -45.2148, -42.6764, -39.3741, -47.6158,
        -52.9541, -50.1207, -48.1770, -48.9691, -44.5730, -45.4469, -44.9750,
        -43.2122, -41.7132, -47.1530, -49.3909, -43.6133, -47.1973, -44.5091,
        -48.9313, -48.9874, -49.7999, -46.9914, -43.9255, -45.9558, -46.7268,
        -46.8406, -44.5671, -42.7406, -49.5476, -46.8566, -41.7708, -42.2490,
        -39.5359, -46.7262, -44.4517, -44.9979, -49.2932, -49.8515, -42.4823,
        -45.5436, -43.9188, -41.7588, -42.5834, -41.6650, -44.7643, -45.8705,
        -50.3985, -47.8573, -46.7541, -36.1047, -44.6856, -49.3110, -47.2863,
        -45.1809, -43.8301, -44.9362, -48.1151, -43.8392, -49.3516, -41.7667,
        -43.7730, -46.6448, -52.6854, -48.0042, -41.3739, -45.0748, -46.1313,
        -44.2312, -43.5201, -42.0362, -47.5573, -49.9541, -42.8470, -43.0699,
        -48.6505, -92.1781], device='cuda:0')
log-likelihood  estimate : tensor(-46.1778, device=